In [1]:
# nb_gen_features.ipynb - Generacion de features para modelos ML
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import date
import logging

# Configurar rutas
BASE = Path("D:/trading")
HIST_PATH = BASE / "data" / "historic"
FEA_PATH = BASE / "data" / "features"
LOG_PATH = BASE / "logs" / f"gen_features_{date.today().isoformat()}.log"

# Crear carpeta de salida si no existe
FEA_PATH.mkdir(parents=True, exist_ok=True)

# Configurar log
logging.basicConfig(filename=LOG_PATH, level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
print("Rutas configuradas correctamente.")


Rutas configuradas correctamente.


In [2]:
archivos = list(HIST_PATH.glob("*.parquet"))
print(f"Total de archivos a procesar: {len(archivos)}")
logging.info(f"Total de archivos a procesar: {len(archivos)}")


Total de archivos a procesar: 50


In [3]:
def calcular_features(df):
    df = df.copy()
    df["retorno"] = df["close"].pct_change()
    df["volatilidad"] = df["retorno"].rolling(window=10).std()
    df["sma_10"] = df["close"].rolling(window=10).mean()
    df["sma_20"] = df["close"].rolling(window=20).mean()
    df["ema_10"] = df["close"].ewm(span=10).mean()
    df["ema_20"] = df["close"].ewm(span=20).mean()
    df["rsi_14"] = calcular_rsi(df["close"], 14)
    df["macd"] = df["close"].ewm(span=12).mean() - df["close"].ewm(span=26).mean()
    df["macd_signal"] = df["macd"].ewm(span=9).mean()
    return df

def calcular_rsi(series, period):
    delta = series.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=period).mean()
    avg_loss = pd.Series(loss).rolling(window=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


In [4]:
for archivo in archivos:
    symbol = archivo.stem.upper()
    try:
        df = pd.read_parquet(archivo)
        df["fecha"] = pd.to_datetime(df["datetime"], errors="coerce")
        df["simbolo"] = symbol
        df_features = calcular_features(df)
        salida = FEA_PATH / f"{symbol}_features.parquet"
        df_features.to_parquet(salida, index=False)
        logging.info(f"OK - {symbol} - {len(df_features)} filas procesadas.")
        print(f"{symbol} listo.")
    except Exception as e:
        logging.error(f"ERROR - {symbol} - {e}")
        print(f"{symbol} fallo.")


AEP listo.
AGNCO listo.
AGNCP listo.
BAC listo.
BTSGU listo.
BX listo.
C listo.
CCI listo.
CFG listo.
COP listo.
CRBG listo.
CTVA listo.
DELL listo.
DGX listo.
DUK listo.
ECCF listo.
ED listo.
EPD listo.
EQR listo.
EW listo.
EXE listo.
FITBO listo.
FWONA listo.
FWONK listo.
HBANL listo.
HBANM listo.
HBANP listo.
HPE listo.
IRM listo.
KHC listo.
KKRS listo.
LPLA listo.
MCHPP listo.
MORN listo.
MRNA listo.
OWL listo.
PCTY listo.
PEN listo.
PRMB listo.
PSTG listo.
RKT listo.
RPRX listo.
SOJC listo.
SOLV listo.
SYM listo.
TBB listo.
TEM listo.
TPG listo.
VLYPN listo.
WPC listo.


In [5]:
# ✅ Validar archivos de features para entrenamiento
import pandas as pd
from pathlib import Path

FEATURES_DIR = Path("D:/trading/data/features")
columnas_minimas = {"fecha", "simbolo", "close"}

archivos = list(FEATURES_DIR.glob("*.parquet"))
errores = []

print(f"Total de archivos encontrados: {len(archivos)}\n")

for file in archivos:
    try:
        df = pd.read_parquet(file)
        if not columnas_minimas.issubset(df.columns):
            errores.append(f"{file.name} columnas faltantes: {columnas_minimas - set(df.columns)}")
        elif df.isna().sum().sum() > 0:
            errores.append(f"{file.name} contiene valores nulos")
        elif df.empty:
            errores.append(f"{file.name} esta vacio")
    except Exception as e:
        errores.append(f"{file.name} error de lectura: {e}")

if errores:
    print("Archivos con errores encontrados:\n")
    for err in errores:
        print(" -", err)
else:
    print("✅ Todos los archivos son validos para entrenamiento.")


Total de archivos encontrados: 50

Archivos con errores encontrados:

 - AEP_features.parquet contiene valores nulos
 - AGNCO_features.parquet contiene valores nulos
 - AGNCP_features.parquet contiene valores nulos
 - BAC_features.parquet contiene valores nulos
 - BTSGU_features.parquet contiene valores nulos
 - BX_features.parquet contiene valores nulos
 - CCI_features.parquet contiene valores nulos
 - CFG_features.parquet contiene valores nulos
 - COP_features.parquet contiene valores nulos
 - CRBG_features.parquet contiene valores nulos
 - CTVA_features.parquet contiene valores nulos
 - C_features.parquet contiene valores nulos
 - DELL_features.parquet contiene valores nulos
 - DGX_features.parquet contiene valores nulos
 - DUK_features.parquet contiene valores nulos
 - ECCF_features.parquet contiene valores nulos
 - ED_features.parquet contiene valores nulos
 - EPD_features.parquet contiene valores nulos
 - EQR_features.parquet contiene valores nulos
 - EW_features.parquet contiene

In [6]:
# Celda para mostrar el log completo del proceso de generacion de features
from pathlib import Path
from datetime import date

# Ruta del log
LOG_PATH = Path("D:/trading/logs") / f"gen_features_{date.today().isoformat()}.log"

# Leer y mostrar contenido
if LOG_PATH.exists():
    with open(LOG_PATH, "r", encoding="utf-8") as f:
        contenido = f.read()
    print(f"Contenido del log generado: {LOG_PATH.name}\n")
    print(contenido)
else:
    print("Log no encontrado.")


Contenido del log generado: gen_features_2025-05-27.log

2025-05-27 22:22:22,960 - INFO - Total de archivos a procesar: 50
2025-05-27 22:22:23,214 - INFO - OK - AEP - 5000 filas procesadas.
2025-05-27 22:22:23,255 - INFO - OK - AGNCO - 1422 filas procesadas.
2025-05-27 22:22:23,294 - INFO - OK - AGNCP - 1334 filas procesadas.
2025-05-27 22:22:23,377 - INFO - OK - BAC - 5000 filas procesadas.
2025-05-27 22:22:23,419 - INFO - OK - BTSGU - 333 filas procesadas.
2025-05-27 22:22:23,479 - INFO - OK - BX - 4511 filas procesadas.
2025-05-27 22:22:23,530 - INFO - OK - C - 5000 filas procesadas.
2025-05-27 22:22:23,581 - INFO - OK - CCI - 5000 filas procesadas.
2025-05-27 22:22:23,616 - INFO - OK - CFG - 2684 filas procesadas.
2025-05-27 22:22:23,683 - INFO - OK - COP - 5000 filas procesadas.
2025-05-27 22:22:23,709 - INFO - OK - CRBG - 676 filas procesadas.
2025-05-27 22:22:23,738 - INFO - OK - CTVA - 1510 filas procesadas.
2025-05-27 22:22:23,790 - INFO - OK - DELL - 2206 filas procesadas.
20

In [7]:
from pathlib import Path

# Listar archivos en la carpeta de logs
log_dir = Path("D:/trading/logs")
log_files = sorted(log_dir.glob("*.log"))

print(f"Se encontraron {len(log_files)} archivos de log:\n")
for f in log_files:
    print("-", f.name)


Se encontraron 17 archivos de log:

- breakout_volumen.log
- cruce_medias.log
- ema_9_21_cruce.log
- ema_pullback.log
- gap_open_strategy.log
- gen_features_2025-05-27.log
- inside_bar_breakout.log
- ma_envelope_reversals.log
- macd_cruce.log
- macd_hist_reversal.log
- mov_avg.log
- rsi_divergencia.log
- rsi_reversion.log
- shp_2025-05-27.log
- sma_10_50_cruce.log
- soporte_resistencia.log
- volatilidad_contraria.log
